In [1]:
import open3d.ml as _ml3d
import open3d as o3d
import open3d.ml.torch as ml3d  # just switch to open3d.ml.tf for tf usage
import os

import warnings
warnings.filterwarnings("ignore")

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
import torch
print(torch.cuda.is_available())

True


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
cfg_file = "configs/kpconv_parislille3d_OPTIMIZEDFORTRAINING.yml"
cfg = _ml3d.utils.Config.load_from_file(cfg_file)

In [5]:
device

device(type='cuda')

In [6]:
dataset = ml3d.datasets.ParisLille3D(dataset_path='/mnt/d/Krishna/Research/Datasets/Paris/', use_cache=True)

#dataset = CustomSemantic3D(dataset_path='/mnt/d/Krishna/Research/Datasets/Semantic3D/semantic_downsampled', use_cache=True)

In [7]:
dataset.get_split_list("train")

['/mnt/d/Krishna/Research/Datasets/Paris//training_10_classes/Lille1_1.ply',
 '/mnt/d/Krishna/Research/Datasets/Paris//training_10_classes/Lille1_2.ply',
 '/mnt/d/Krishna/Research/Datasets/Paris//training_10_classes/Paris.ply']

In [8]:
model = ml3d.models.KPFCNN(**cfg.model)

In [9]:
pipeline = ml3d.pipelines.SemanticSegmentation(model=model, dataset=dataset, device="gpu", **cfg.pipeline)

In [10]:
import os

ckpt_folder = "pretrained/"
os.makedirs(ckpt_folder, exist_ok=True)
ckpt_path = ckpt_folder + "kpconv_parislille3d_202011241550utc.pth"

In [11]:
'''
ADDED ENTRIES

num_workers: 0
pin_memory: False

TO THE yml FILE IN UNDER THE PIPELINE SECTION FOR THE RuntimeError OCCURED DUE TO COMPUTE CONSTRAINTS
'''

'\nADDED ENTRIES\n\nnum_workers: 0\npin_memory: False\n\nTO THE yml FILE IN UNDER THE PIPELINE SECTION FOR THE RuntimeError OCCURED DUE TO COMPUTE CONSTRAINTS\n'

## Training from scratch

In [12]:
%%time

pipeline.run_train()

validation: 100%|█████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.53s/it]


CPU times: user 4min 1s, sys: 53.5 s, total: 4min 55s
Wall time: 13min 2s


## Retraining using checkpoints

In [ ]:
%%time

pipeline.load_ckpt(ckpt_path=ckpt_path, is_resume=True)
pipeline.run_train()

### Inference

Testing directly on IIITB data

In [6]:
cfg_file_1 = "configs/kpconv_parislille3d_OPTIMIZEDFORTRAINING.yml"
cfg_1 = _ml3d.utils.Config.load_from_file(cfg_file_1)

cfg_1.dataset['dataset_path'] = '/mnt/d/Krishna/Research/Datasets/Paris/'
tDataset = ml3d.datasets.ParisLille3D(cfg_1.dataset.pop('dataset_path', None), **cfg_1.dataset)
#pipeline = ml3d.pipelines.SemanticSegmentation(model, dataset=dataset, device="gpu", **cfg.pipeline)

# download the weights.
#ckpt_folder = "logs/KPFCNN_Semantic3D_torch/checkpoint/"
#os.makedirs(ckpt_folder, exist_ok=True)
#ckpt_path = ckpt_folder + "ckpt_05000.pth"

""" COMMENT THIS!!!!!!!!!!!
model = ml3d.models.KPFCNN(**cfg_1.model)

pipeline = ml3d.pipelines.SemanticSegmentation(model, dataset=tDataset, device="gpu", **cfg_1.pipeline)

ckpt_folder = "logs/KPFCNN_ParisLille3D_torch/checkpoint/"
os.makedirs(ckpt_folder, exist_ok=True)
ckpt_path = ckpt_folder + "ckpt_00500.pth"

pipeline.load_ckpt(ckpt_path=ckpt_path)
"""

In [7]:
# load the parameters.
#pipeline.load_ckpt(ckpt_path=ckpt_path)

In [14]:
"""dataset.get_split_list("test")
test_split = dataset.get_split("test")
dataTest = test_split.get_data(5)"""

In [8]:
tDataset.get_split_list("test")

['/mnt/d/Krishna/Research/Datasets/Paris//test_10_classes/000003.ply',
 '/mnt/d/Krishna/Research/Datasets/Paris//test_10_classes/ajaccio_2.ply',
 '/mnt/d/Krishna/Research/Datasets/Paris//test_10_classes/ajaccio_57.ply',
 '/mnt/d/Krishna/Research/Datasets/Paris//test_10_classes/dijon_9.ply',
 '/mnt/d/Krishna/Research/Datasets/Paris//test_10_classes/IIITB_base_Readable.ply',
 '/mnt/d/Krishna/Research/Datasets/Paris//test_10_classes/IIITB_ParisLille_GT_BINARY.ply']

In [9]:
test_split = tDataset.get_split("test")
dataTest = test_split.get_data(5)

In [15]:
result = pipeline.run_inference(dataTest)

test 0/1: 100%|█████████████████████████████████████████████████████████████████| 442954/442954 [05:07<00:00, 81.92it/s]

In [16]:
print(result)

{'predict_labels': array([1, 1, 1, ..., 1, 1, 1]), 'predict_scores': array([[0.02333  , 0.03836  , 0.000936 , ..., 0.000889 , 0.004147 ,
        0.02516  ],
       [0.02747  , 0.03546  , 0.0009317, ..., 0.000982 , 0.00237  ,
        0.02574  ],
       [0.03111  , 0.05386  , 0.002213 , ..., 0.002007 , 0.00615  ,
        0.03705  ],
       ...,
       [0.04504  , 0.04718  , 0.001527 , ..., 0.001439 , 0.004547 ,
        0.03607  ],
       [0.0412   , 0.0475   , 0.001935 , ..., 0.001598 , 0.00478  ,
        0.03745  ],
       [0.039    , 0.04922  , 0.001978 , ..., 0.001653 , 0.005466 ,
        0.03714  ]], dtype=float16)}


In [17]:
len(result['predict_labels'])

2273624

In [18]:
set(result['predict_labels'])

{0, 1, 7, 8}

In [51]:
newData = dataTest

In [52]:
newData['label'] = result['predict_labels']

In [53]:
import numpy as np

# Assuming newData is already defined as in your example

# Combine all data into a single array for writing to a text file
# Structure: x, y, z, r, g, b, label
data_to_write = np.hstack((
    newData['point'],                       # x, y, z coordinates
    newData['feat'][:, :3],                 # r, g, b colors
    newData['label'].reshape(-1, 1)         # label as a single column
))

# Save to a text file
np.savetxt("output_Paris_KPConv_1000_retrain.txt", data_to_write, fmt='%.6f %.6f %.6f %.3f %.3f %.3f %d')

print("Point cloud data with labels saved to output_Paris_KPConv_1000_retrain.txt")

TypeError: 'NoneType' object is not subscriptable

test 0/1: 100%|█████████████████████████████████████████████████████████████████| 442954/442954 [05:10<00:00, 19.64it/s]